# Welcome to the party, pal

In [ ]:
import csv, json
import pandas as pd

from elasticsearch import Elasticsearch # https://elasticsearch-py.readthedocs.io/en/v8.1.1/
from elasticsearch.helpers import bulk

In [ ]:
# fill in cloud id and user details
client = Elasticsearch(
    cloud_id="My_deployment:ZXVyb3BlLW5vcnRoMS5nY3AuZWxhc3RpYy1jbG91ZC5jb20kYmRlOTEwODNiNGJiNGM1NGJlMjE3NmUyZDYwODI0NTMkOTY2MGVlYmQ4NzJmNGUxOWFkYjZmNGJlNGVmYTVjNjc=",
    basic_auth=("elastic", "xeygCBUwpiekHhkwynMDInr4"),
)
# sanity check
health = client.cluster.health()
print(health)
assert health['status'] == "green"

## Let's ingest some data

In [ ]:
movie_file = 'movie_metadata.csv'

df = pd.read_csv(movie_file, header=0, sep=',', skipinitialspace = True, )

# mild wrangling
df.dropna(how='all', inplace=True)
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
split_cols = ['genres', 'plot_keywords']
for col in split_cols:
    df[col] = df[col].fillna('').transform(lambda s: s.split('|'))


def gen_data(index, data):
    for d in data:
        yield {
            '_index': index,
            '_source': d
        }

index = 'my_movies'
data = json.loads(df.to_json(orient='records', date_format=None, date_unit='s', force_ascii=False))

bulk(client, gen_data(index, data), max_retries=10, )

## ... and find it

In [ ]:
# some movie named avatar
client.search(index=index, query={
        'match': {
            'movie_title': 'Avatar'
        }
    }
)

In [ ]:
client.search(index=index, size=0, aggs={
    "movie-languages": {
      "terms": {
        "field": "language.keyword",
        "size": 10
      }
    }
  }
)